# Exploring Cross-Validation

This script will walk you through the process of fitting a linear model using polynomial basis functions, and the selection of a hyper-parameter using a validation data set.

# Data

The data is generated using the following model: 
$$ y = f(x) + \epsilon = 5x(x-0.5)(x-1) + \epsilon$$
where $\epsilon \sim N(0,0.01)$ is a Gaussian noise term with zero mean and standard deviation equal to 0.1.

The training and test data have already been generated for you and the section below shows you how to load them and what they look like.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Loading training and test data
x_train = np.loadtxt('Data/x_train.csv',delimiter=',')
y_train = np.loadtxt('Data/y_train.csv',delimiter=',')
x_test = np.loadtxt('Data/x_test.csv',delimiter=',')
y_test = np.loadtxt('Data/y_test.csv',delimiter=',')

# Plotting data purely for verification
plt.plot(x_train,y_train,'k.',x_test,y_test,'r.')
plt.xlabel('x')
plt.ylabel('y')
plt.legend({'Training','Testing'})
plt.show()

# Fitting a Polynomial to the Data

The section below shows how to do a simple fit for a quadratic polynomial.

In [ ]:
import polyfit as pf

# Fitting model
deg = 2
beta = pf.fit(x_train,y_train,deg)

# Computing training error
y_train_pred = pf.predict(x_train,beta)
err = pf.rmse(y_train,y_train_pred)
print('Training Error = {:2.3}'.format(err))

# Computing test error
y_test_pred = pf.predict(x_test,beta)
err = pf.rmse(y_test,y_test_pred)
print('Test Error = {:2.3}'.format(err))

# Plotting fitted model
x = np.linspace(0,1,100)
y = pf.predict(x,beta)
plt.plot(x,y,'b-',x_train,y_train,'ks',x_test,y_test,'rs')
plt.legend(['Prediction','Training Points','Test Points'])
plt.show()

# Hyper-Parameter Selection

This section illustrates how to perform hyper-parameter selection where the capacity of the model is captured by the degree of the polynomial used for fitting.

First, we fit the data to the entire training set and compute the corresponding training and test errors. We used all of the data since we are not performing any hyper-parameter selection at this point.

## Question 1 [2 pts]

Your first tasks is to split the data into pre-val training and validation for a K-fold cross validation. You should complete the function within the provided script. Keep all measurements in the same order as the original training set.

In [ ]:
import kfold_split as ks
import importlib
importlib.reload(ks)

# This is the total number of folds
K = 10

# This is the function that needs to be modified within the specified block in the script.
# Note that the last entry is the current k-fold splid to be returned. That is, passing a
# value of 0 will return the first fold split with the first fold as the validation set.
x_preval, y_preval, x_val, y_val = ks.kfold_split(x_train,y_train,K,0)

print('Dimensions of Validation Set: {}'.format(x_val.shape))
print('Dimensions of Pre-Validation Set: {}'.format(x_preval.shape))

## Question 2 [4 pts]

Compute pre-validation training and validation errors for each of the listed degrees. The training error should show a decreasing pattern. The validation error should decrease and then increase. You should complete the function within the provided script.

In [ ]:
import kfold_crossval as kc
importlib.reload(kc)

# List of degrees considered for the analysis
degList =  np.array([0,1,2,3,4,5,6,7,8,9,10])

# This is the total number of folds
K = 10

# Getting arrays with the errors for all folds. We keep track of all errors so
# the mean and standard deviation can be computed below.
errPreVal, errVal = kc.kfold_crossval(x_train,y_train,degList,K)

# Computing means and standard deviation for the errors
errPreVal_mean = np.mean(errPreVal,axis=1)
errVal_mean = np.mean(errVal,axis=1)
errVal_std = np.std(errVal,axis=1)

# Plotting results
plt.plot(degList,errPreVal_mean,'b.-',degList,errVal_mean,'r.-',degList,errVal_mean+errVal_std,'r--')
plt.xlabel('degree')
plt.ylabel('RMSE')
plt.legend(['Mean Pre-Val Training Error','Mean Val Error','Mean Val Error + Std'])
plt.show()

# Performance of Optimal Model

We begin by selecting the optimal degree using this rule: *Select the most parsimonious model that has a cross-validation value within a standard deviation from the best model $\theta^*$.*

In [ ]:
# Getting the index of the degree with minimum mean validation error
idx = np.argmin(errVal_mean)

# Getting list of degrees that are below the mininum error plus the std
tmp = degList[errVal_mean<errVal_mean[idx]+errVal_std[idx]]

# The optimal degree is the minimum of the previous list
degOpt = min(tmp)

print('Optimal Degree = {}'.format(degOpt))
print('Mean Validation Error = {:.3f} +/- {:.3f}'.format(errVal_mean[idx],errVal_std[idx]))

Next, we fit a model with all the training data and show its performance.

In [ ]:
# Fitting the model
beta = pf.fit(x_train,y_train,degOpt)
y_test_pred = pf.predict(x_test,beta)
errTest = pf.rmse(y_test,y_test_pred)

# Displaying the test error
print('Test Error = {:.3f}'.format(errTest))

# Plotting fitted model
x = np.linspace(0,1,100)
y = pf.predict(x,beta) # Use the beta from the full training set for better visualization
plt.plot(x,y,'b-',x_train,y_train,'ks',x_test,y_test,'rs')
plt.legend(['Prediction','Training Points','Test Points'])
plt.show()